In [ ]:
# PEG backtesting

In [ ]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEG import main

In [ ]:
import pandas as pd

# Initial setup
initial_cash = 10000
portfolio_cash = initial_cash
portfolio_stock = 0  # initially no stock
buy_price = 0
trade_fee_percent = 0.001  # 0.1%
profit_target = 1.10  # 10% profit target
stop_loss = 0.95  # 5% stop loss
alert_column = 'alert'  # column in your DataFrame with 1s and 0s for potential trading opportunity

# Metrics initialization
total_trades = 0
total_profit = 0
max_profit = 0
max_drawdown = 0
peak_value = initial_cash

# Assuming your DataFrame has columns: 'date', 'ticker', 'close', 'alert' and it's sorted by date for each ticker
def backtest(df):
    global portfolio_cash, portfolio_stock, buy_price, total_trades, total_profit, max_profit, max_drawdown, peak_value
    
    trades = []  # To store trade data (buy/sell)
    
    for index, row in df.iterrows():
        close_price = row['close']
        alert = row[alert_column]
        
        # Buy condition (alert = 1, enough cash)
        if alert == 1 and portfolio_cash > 0:
            # Calculate the number of shares to buy
            shares_to_buy = portfolio_cash / close_price
            total_cost = shares_to_buy * close_price
            fee = total_cost * trade_fee_percent
            total_cost += fee  # including fee
            
            # Buy the stock
            portfolio_stock += shares_to_buy
            portfolio_cash -= total_cost
            buy_price = close_price
            
            trades.append({'date': row['date'], 'ticker': row['ticker'], 'type': 'buy', 'price': close_price, 'shares': shares_to_buy, 'fee': fee})
            total_trades += 1
        
        # Check if we should sell based on profit target or stop loss
        if portfolio_stock > 0:
            sell_condition_profit = close_price >= buy_price * profit_target
            sell_condition_loss = close_price <= buy_price * stop_loss
            
            if sell_condition_profit or sell_condition_loss:
                total_value = portfolio_stock * close_price
                fee = total_value * trade_fee_percent
                total_value -= fee  # after fee
                
                # Sell the stock
                profit = total_value - (portfolio_stock * buy_price) - (portfolio_stock * buy_price * trade_fee_percent)
                portfolio_cash += total_value
                total_profit += profit
                portfolio_stock = 0
                
                trades.append({'date': row['date'], 'ticker': row['ticker'], 'type': 'sell', 'price': close_price, 'shares': portfolio_stock, 'fee': fee})
                
                # Calculate max profit and drawdown
                current_value = portfolio_cash
                if current_value > peak_value:
                    peak_value = current_value
                drawdown = (peak_value - current_value) / peak_value
                max_drawdown = max(max_drawdown, drawdown)
                max_profit = max(max_profit, profit)
    
    # Calculate final portfolio value and returns
    final_portfolio_value = portfolio_cash
    total_return = (final_portfolio_value - initial_cash) / initial_cash * 100  # in %
    
    return {
        'trades': trades,
        'final_portfolio_value': final_portfolio_value,
        'total_return': total_return,
        'total_profit': total_profit,
        'total_trades': total_trades,
        'max_profit': max_profit,
        'max_drawdown': max_drawdown
    }

# Example DataFrame setup
# df = pd.DataFrame({
#     'date': [...],  # replace with your data
#     'ticker': [...],  # replace with your data
#     'close': [...],  # replace with your data
#     'alert': [...],  # replace with your calculated alerts
# })

# Run backtest
results = backtest(df)

# Convert trades to DataFrame for analysis
trades_df = pd.DataFrame(results['trades'])

# Output results
print(f"Final portfolio cash: {results['final_portfolio_value']}")
print(f"Total return: {results['total_return']}%")
print(f"Total profit: {results['total_profit']}")
print(f"Total trades: {results['total_trades']}")
print(f"Max profit from a single trade: {results['max_profit']}")
print(f"Max drawdown: {results['max_drawdown'] * 100}%")
